In [22]:
# Dependencies
import tweepy
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Twitter API Keys
from config import (twt_consumer_key, 
                    twt_consumer_secret, 
                    twt_access_token, 
                    twt_access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(twt_consumer_key, twt_consumer_secret)
auth.set_access_token(twt_access_token, twt_access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [23]:
# def get_sentiments(handle, pages):
handle = "@CNN"

compound_list = []

for i in range(1):
    all_tweets = api.user_timeline(handle, page=i)
    for tweet in all_tweets:
        tweet_text = tweet["text"] 
        print(tweet_text)
        sentiment = analyzer.polarity_scores(tweet_text)

        # Add each value to the appropriate list
        compound_list.append(sentiment["compound"])
        
# return compound_list


TweepError: [{'message': 'Internal error', 'code': 131}]

In [ ]:
outlets = ["@CNN","@BBCNews","@CBSNews","@FoxNews","@nytimes"]
sentiments = []
avg_sentiments = []

for outlet in outlets:
    sentiment = get_sentiment(outlet, 5)
    sentiments.append(sentiment)
    avg_sentiment = np.mean(sentiment)
    avg_sentiments.append(avg_sentiment)    

In [ ]:
tweets_ago = range(1,101)

for sentiment_list in sentiments:
    plt.scatter(tweets_ago,sentiment_list)
    
plt.legend(outlets)    
plt.show()

In [ ]:
plt.bar(avg_sentiments)
plt.xlabels(outlets)
plt.xticks()
